In [48]:
from os.path import join
import numpy as np
import pandas as pd
from feature.dataset import make_dataset
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from category_encoders import *
from sklearn.model_selection import StratifiedKFold
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
import lightgbm as lgb


In [66]:
train = pd.read_parquet('./jeju_data/train_48_T40.parquet')
test = pd.read_parquet('./jeju_data/test_48_T40.parquet')
X = train.drop(["target", "tour_count"], axis=1)
test = test.drop(["tour_count"], axis=1)
y = train["target"]

In [72]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

params = {'n_estimators': 4693, 'max_depth': 15, 'min_child_weight': 4, 'gamma': 1, 'learning_rate': 0.018, 'colsample_bytree': 0.9015018647603987, 'lambda': 1.386941960727803, 'alpha': 0.10534800837686535, 'subsample': 1.0}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

XGB_model= {}

for f in range(5):
      print(f'===================================={f+1}============================================')
      train_idx, val_idx = folds[f]
      
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]
      
      XGB = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0)
      XGB.fit(x_train, y_train)
      
      y_pred = XGB.predict(x_val)
      mae = mean_absolute_error(y_val, y_pred)
      print(f"{f + 1} Fold MAE = {mae}")
      XGB_model[f] = XGB
      print(f'================================================================================\n\n')
              


sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

for fold in range(5):
    sample_submission['target'] += XGB_model[fold].predict(test)/5    
    

c:\mb_job\project\Competition\DACON\jeju_traffic_prediction\jeju_project\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


====================================1============================================
1 Fold MAE = 2.9497048628987437


====================================2============================================
2 Fold MAE = 2.9489886860722594


====================================3============================================
3 Fold MAE = 2.9551123687625473


====================================4============================================
4 Fold MAE = 2.9505465951278285


====================================5============================================
5 Fold MAE = 2.952824939747864




In [74]:
sample_submission.to_csv("./xgb_fold_mb.csv", index=False)

In [75]:
df_imp = pd.DataFrame({'imp':XGB.feature_importances_}, index = XGB.feature_names_in_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp

,imp
day_of_week,0.000598
post_holiday,0.000626
sin_time,0.000783
group_time,0.000807
rest,0.000810
cos_time,0.000868
pre_holiday,0.000876
holiday,0.000876
base_hour,0.000939
vacation,0.000942


In [2]:
import pandas as pd
a = pd.read_csv('xgb_fold_mb.csv')
a

,id,target
0,TEST_000000,26.238368
1,TEST_000001,42.895396
2,TEST_000002,67.753142
3,TEST_000003,38.073544
4,TEST_000004,42.974990
...,...,...
291236,TEST_291236,47.524633
291237,TEST_291237,51.298343
291238,TEST_291238,22.505252
291239,TEST_291239,22.617785


In [3]:
a['target'] = round(a['target'])


In [5]:
a.to_csv("./xgb_fold_mb_round.csv", index=False)